In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import gradio as gr

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <61623A3D-DA3C-3AAD-B2F0-D363151DDB3F> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <EACD001F-FCB9-380E-AD73-D522177FC040> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [4]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "tiiuae/falcon-7b-instruct"  # Corrected model name

# Set up offload folder to store parts of the model that won't fit in GPU memory
offload_folder = "./offload"  
os.makedirs(offload_folder, exist_ok=True)

# Load the Falcon model with offloading
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cpu",
    offload_folder=offload_folder,  # Specify the offload folder for offloaded weights
    torch_dtype=torch.float16  # Using mixed precision to reduce memory load
)

# The rest of your model code follows...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
def test_llm_model():
    prompt = "What is artificial intelligence?"
    input_ids = tokenizer.encode(prompt, return_tensors="pt")  # Tokenize the input prompt

    # Create an attention mask
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)  # Create a mask of ones since there's no padding in the input

    # Generate text using the model
    with torch.no_grad():  # No gradients needed for inference
        generated_output = model.generate(
            input_ids,
            attention_mask=attention_mask,  # Provide the attention mask to avoid warnings
            max_length=100,  # Limit response length for test purposes
            do_sample=True,  # Enable sampling for variety
            temperature=0.7,  # Control creativity
            pad_token_id=tokenizer.eos_token_id  # Set pad token ID explicitly
        )
    
    # Decode and print the output
    output_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    print("Generated Response:\n", output_text)

# Run the test function to see if the model is working
test_llm_model()

In [ ]:
import gradio as gr

# Function to polish the resume
def polish_resume(position_name, resume_content, polish_prompt=""):
    if polish_prompt and polish_prompt.strip():
        prompt_use = f"Given the resume content: '{resume_content}', polish it based on the following instructions: {polish_prompt} for the {position_name} position."
    else:
        prompt_use = f"Suggest improvements for the following resume content: '{resume_content}' to better align with the requirements and expectations of a {position_name} position. Return the polished version, highlighting necessary adjustments for clarity, relevance, and impact in relation to the targeted role."

    generated_response = model.generate(
        tokenizer.encode(prompt_use, return_tensors="pt"),
        max_length=512,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7
    )
    return tokenizer.decode(generated_response[0], skip_special_tokens=True)

resume_polish_application = gr.Interface(
    fn=polish_resume,
    inputs=[
        gr.Textbox(label="Position Name", placeholder="Enter the name of the position..."),
        gr.Textbox(label="Resume Content", placeholder="Paste your resume content here...", lines=20),
        gr.Textbox(label="Polish Instruction (Optional)", placeholder="Enter specific instructions or areas for improvement (optional)...", lines=2),
    ],
    outputs=gr.Textbox(label="Polished Content"),
    title="Resume Polish Application",
    description="This application helps you polish your resume. Enter the position you want to apply, your resume content, and specific instructions or areas for improvement (optional), then get a polished version of your content."
)

resume_polish_application.launch()


In [ ]:
def generate_cover_letter(position_name, company_name, personal_experience, key_skills=""):
    prompt = (
        f"I am applying for the position of {position_name} at {company_name}. "
        f"Below are my relevant experiences: {personal_experience}. "
    )
    if key_skills and key_skills.strip():
        prompt += f"Additionally, I possess the following key skills that are valuable for this role: {key_skills}. "
    prompt += "Please generate a compelling cover letter based on this information that highlights my skills, experiences, and interest in the company."


    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    with torch.no_grad(): 
        generated_output = model.generate(
            input_ids,
            attention_mask=attention_mask, 
            max_length=512,  
            do_sample=True, 
            temperature=0.7,  
            pad_token_id=tokenizer.eos_token_id  
        )
    
   
    output_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    return output_text


cover_letter_app = gr.Interface(
    fn=generate_cover_letter,
    inputs=[
        gr.Textbox(label="Position Name", placeholder="Enter the name of the position..."),
        gr.Textbox(label="Company Name", placeholder="Enter the name of the company..."),
        gr.Textbox(label="Personal Experience", placeholder="Describe your relevant experiences...", lines=5),
        gr.Textbox(label="Key Skills (Optional)", placeholder="Enter any specific skills you want to highlight (optional)...", lines=2),
    ],
    outputs=gr.Textbox(label="Generated Cover Letter"),
    title="Cover Letter Generator",
    description="This application helps you generate a cover letter based on the position, company, your relevant experiences, and any key skills you want to highlight. Enter the required information and get a tailored cover letter."
)


cover_letter_app.launch()